In [1]:
# !pip install pyarrow
import pandas as pd
import pyarrow.parquet as pq
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
DATA_DIR = '../data/'
http_bots = pq.read_table(DATA_DIR + 'botsHTTPRequests-20180216_1416GMT.parquet').to_pandas()
http_users = pq.read_table(DATA_DIR + 'usersHTTPRequests-20180216_1416GMT.parquet').to_pandas()
#http_bots = http_bots.sample(frac = 0.03)
#http_users = http_users.sample(frac=0.03)
http_users['target'] = 0
http_bots['target'] = 1

http = pd.concat([http_bots, http_users])

http.head()

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,userAgent,userId,hour,target
0,8.293748e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518791752182,None,https://ok.ru/web-api/pts/postingFormJs,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,0hs35/dB/t+uT7u/c71+Vw==,17,1
1,8.293748e+18,friendMain,3rRnET9ABG3feVMANINXpg==,POST,friendAltGroup,https://www.ok.ru/,NAV,1518793194690,friendAltGroup,https://www.ok.ru/profile/561514539517/groups,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,HW3KxEEODcSX6gJg1YZQtQ==,17,1
2,7.895490e+17,None,Qsv9DECmtHgfUZ0GAVUoLQ==,POST,PinsEditAjaxRB,https://ok.ru/,REQ,1518791913138,None,https://ok.ru/profile/589603942951/statuses,Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537....,e9iJCewg5Ww1ijKmYKdAyg==,17,1
3,8.293748e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518792629926,None,https://ok.ru/web-api/pts/webpush,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,HW3KxEEODcSX6gJg1YZQtQ==,17,1
4,-8.322615e+18,None,XK5THOHUVJHpqGojQAKrMQ==,GET,userMain,None,NAV,1518795472623,userMain,https://ok.ru/,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,f3Z8Sr501bB3SvEgCo+9Rg==,18,1


In [3]:
http = http.sample(frac=1).reset_index(drop=True)

In [4]:
http.head()

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,userAgent,userId,hour,target
0,-2.085537e+18,None,07NtqPyZzxTvxexa/uOnmA==,POST,SuggestStickers,https://ok.ru/,REQ,1518793107257,None,https://ok.ru/dk,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,ehCYYCNR5p5CadOkzr4WIA==,17,0
1,7.693487e+18,None,7bMFBcnTgBAhMUwabngpKQ==,POST,FourthCol,https://ok.ru/,REQ,1518791867438,None,https://ok.ru/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,QgCLyEWWtx6lHKXaxNsS/g==,17,0
2,-5.216583e+18,None,WCQhM0EGeDY2DROuhFwHlw==,POST,spring-mvc,https://ok.ru/,REQ,1518794957468,None,https://ok.ru/web-api/messages/conversation/ma...,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,2mtvC8JfdJW49iLdtHKztg==,18,0
3,6.958212e+16,None,/9A9JZ+Ao84mVOnpX/3vMQ==,POST,PopLayerPhoto,https://ok.ru/,REQ,1518795265214,None,https://ok.ru/feed,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,JMd86zyKJRqEIUMufDfXmA==,18,0
4,-6.347539e+17,None,i8uvTGkZHM5kxsYYDcZRhg==,POST,videoStatNew,https://ok.ru/,REQ,1518794482457,None,https://ok.ru/dk,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,Dz1Q3qr++0M1UqUbLp0r8g==,18,0


In [5]:
print('Кол-во уникальных операций: {}'.format(len(http['operation'].unique())))

Кол-во уникальных операций: 480


### Приведем категориальный столбей общей выборки к разряженному виду и разделим выборку на train и test

Сначала закодируем названия операция чиселками

In [6]:
operations = http['operation']

dict_code_op = {}

c = 0
for op in operations.unique():
    dict_code_op[op] = c
    c += 1
    
operations = operations.apply(lambda x: dict_code_op[x])    
#operations = pd.DataFrame()

In [7]:
operations_sparse = csr_matrix(([1] * operations.shape[0],
                                operations,
                                range(0, operations.shape[0] + 1, 1)))[:, 1:]
#operations_sparse.toarray()

In [8]:
# С помощью train_test_split разбивать не получается, 
# тк разряженный категориальный стобец требует индекса
# и не хочется преобразовывать csr_matrix к pd.DataFrame

# X_train, X_holdout, y_train, y_holdout = train_test_split(
#     http.drop('target', axis=1), http['target'], test_size=0.3, random_state=17)

# operations_sparse -> X_train/X_test; http.target -> y_train/y_test
idx_split = int(http.shape[0] * 0.7)
X_train = operations_sparse[:idx_split]
X_holdout = operations_sparse[idx_split:] #X_test = operations_sparse[idx_split:, :] для df
y_train = http['target'][:idx_split]
y_holdout = http['target'][idx_split:]

### Обучим модель

In [9]:
%%time
# Посмотрим вообще как долго обучаться

logit = LogisticRegression(C=0.1, n_jobs=-1, random_state=17)
logit.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


CPU times: user 1.82 s, sys: 15.1 ms, total: 1.83 s
Wall time: 1.83 s


In [17]:
holdout_pred = logit.predict_proba(X_holdout)

In [18]:
holdout_pred.shape

(303114, 2)

In [19]:
holdout_pred

array([[0.99386551, 0.00613449],
       [0.99764369, 0.00235631],
       [0.99230277, 0.00769723],
       ...,
       [0.9984107 , 0.0015893 ],
       [0.9984107 , 0.0015893 ],
       [0.9981742 , 0.0018258 ]])

In [20]:
n_bots = 0
for i in holdout_pred:
    if i[0] < i[1]:
        n_bots += 1
print('Грубая оценка кол-ва ботов: {} из {}'.format(n_bots, http.shape[0]))

Грубая оценка кол-ва ботов: 215 из 1010377


In [23]:
roc_auc_score(y_holdout, holdout_pred[:, 1])

0.9091452918277505